Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [5]:
batch_size = 128
num_hidden_nodes = 1024   
num_layers = 2
beta = 0.01

def apply_layers(input, weights, biases, num_layers, dropout = False):
    output = input
    for i in range(num_layers):
        output = tf.matmul(output, weights[i]) + biases[i]
        if i < (num_layers - 1):
            if dropout:
                output = tf.nn.dropout(tf.nn.relu(output), 0.2)
            else:
                output = tf.nn.relu(output)

    return output        

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = []  
  weights.append(tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes])))
  weights.append(tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels])))
 
  biases = []
  biases.append(tf.Variable(tf.zeros([num_hidden_nodes])))
  biases.append(tf.Variable(tf.zeros([num_labels])))  
            
  
  # Training computation.
  logits = apply_layers(tf_train_dataset, weights, biases, num_layers, dropout = True)  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + \
    beta * (tf.nn.l2_loss(weights[0]) + tf.nn.l2_loss(weights[1])))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = apply_layers(tf_valid_dataset, weights, biases, num_layers)  
  test_prediction = apply_layers(tf_test_dataset, weights, biases, num_layers)  

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size) #256 for overfitting
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3943.601318
Minibatch accuracy: 8.6%
Validation accuracy: 28.1%
Minibatch loss at step 500: 23.565868
Minibatch accuracy: 73.4%
Validation accuracy: 82.4%
Minibatch loss at step 1000: 1.198762
Minibatch accuracy: 78.1%
Validation accuracy: 82.6%
Minibatch loss at step 1500: 0.966501
Minibatch accuracy: 78.9%
Validation accuracy: 82.5%
Minibatch loss at step 2000: 1.073093
Minibatch accuracy: 80.5%
Validation accuracy: 82.0%
Minibatch loss at step 2500: 1.040205
Minibatch accuracy: 76.6%
Validation accuracy: 82.2%
Minibatch loss at step 3000: 1.072249
Minibatch accuracy: 71.9%
Validation accuracy: 82.9%
Test accuracy: 89.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [131]:
batch_size = 512
num_hidden_nodes = [2048, 1024, 512]
num_layers = 4
beta = 0.01

def apply_layers(input, weights, biases, num_layers, dropout = False):
    output = input
    for i in range(num_layers):
        output = tf.matmul(output, weights[i]) + biases[i]
        if i < (num_layers - 1):
            if dropout:
                output = tf.nn.dropout(tf.nn.relu(output), 0.5)
            else:
                output = tf.nn.relu(output)

    return output        

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = []  
  weights.append(tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes[0]], stddev=0.05)))
  for i in range(num_layers - 2):
    weights.append(tf.Variable(
      tf.truncated_normal([num_hidden_nodes[i], num_hidden_nodes[i + 1]], stddev=0.05)))
  weights.append(tf.Variable(
    tf.truncated_normal([num_hidden_nodes[-1], num_labels], stddev=0.1)))
 
  biases = []
  for i in range(num_layers - 1):
    biases.append(tf.Variable(tf.zeros([num_hidden_nodes[i]])))
  biases.append(tf.Variable(tf.zeros([num_labels])))  
#    biases.append(tf.Variable(tf.constant(0.1, shape=[num_hidden_nodes])))
#  biases.append(tf.Variable(tf.constant(0.1, shape=[num_labels])))  

  print([w.shape for w in weights])
  print([b.shape for b in biases])
            
  
  # Training computation.
  logits = apply_layers(tf_train_dataset, weights, biases, num_layers, dropout = True)  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + \
    beta * sum([tf.nn.l2_loss(w) for w in weights]))
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.0001, global_step, 6000, 0.9)
#  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
#  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
#  optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)
#  optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)
  optimizer = tf.train.AdagradOptimizer(1e-3).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = apply_layers(tf_valid_dataset, weights, biases, num_layers)
  test_prediction = apply_layers(tf_test_dataset, weights, biases, num_layers)

[TensorShape([Dimension(784), Dimension(2048)]), TensorShape([Dimension(2048), Dimension(1024)]), TensorShape([Dimension(1024), Dimension(512)]), TensorShape([Dimension(512), Dimension(10)])]
[TensorShape([Dimension(2048)]), TensorShape([Dimension(1024)]), TensorShape([Dimension(512)]), TensorShape([Dimension(10)])]


In [132]:
import numpy as np

num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size) #256 for overfitting
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
#    indexes = np.random.choice(range(train_labels.shape[0]), size=batch_size)
#    batch_data = train_dataset[indexes, :]
#    batch_labels = train_labels[indexes, :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 46.173180
Minibatch accuracy: 10.4%
Validation accuracy: 10.1%
Minibatch loss at step 500: 41.184162
Minibatch accuracy: 52.9%
Validation accuracy: 76.9%
Minibatch loss at step 1000: 39.721943
Minibatch accuracy: 62.5%
Validation accuracy: 78.8%
Minibatch loss at step 1500: 38.344185
Minibatch accuracy: 68.4%
Validation accuracy: 80.1%
Minibatch loss at step 2000: 37.229485
Minibatch accuracy: 68.8%
Validation accuracy: 80.6%
Minibatch loss at step 2500: 35.959309
Minibatch accuracy: 73.4%
Validation accuracy: 81.0%
Minibatch loss at step 3000: 34.916382
Minibatch accuracy: 73.2%
Validation accuracy: 81.3%
Minibatch loss at step 3500: 33.779442
Minibatch accuracy: 75.4%
Validation accuracy: 81.5%
Minibatch loss at step 4000: 32.806389
Minibatch accuracy: 76.2%
Validation accuracy: 81.7%
Minibatch loss at step 4500: 31.709845
Minibatch accuracy: 78.3%
Validation accuracy: 81.9%
Minibatch loss at step 5000: 30.805756
Minibatch accuracy: 77.9%
Validat